<a href="https://colab.research.google.com/github/ShinAsakawa/2019cnps/blob/master/notebooks/2019cnps_addtion_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# エンコーダ・デコーダモデル (seq2seq) による 足し算のデモ


* 入力: "535+61"
* 出力 "596"
* 埋め草文字として空白の繰り返しを用いる

* 入力はオプションで反転させることが可能。
* 翻訳モデルなどでは，入力系列を反転させることで，性能向上が認められることが知られている。

## 文献
1. [Learning to Execute](http://arxiv.org/abs/1410.4615)
2. [Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)

## 結果

理論的には、ソースとターゲットの間に短期的な依存関係が導入される。
以下実行結果

* 2 桁 反転 足し算
    + 1 層 LSTM 99% 正解率

*  3 桁 反転 足し算
    + 1 層 LSTM 99% 正解率

* 4 桁 反転 足し算。
    + 1 層 LSTM 99% 正解率

* 5 桁 反転。
    + 1 層 LSTM 99%


In [15]:
from keras.models import Sequential
from keras import layers
import numpy as np
import keras
print(f'keras.__version__:{keras.__version__}')

keras.__version__:3.8.0


In [3]:
class CharacterTable(object):
    """任意の文字集合が問題として与えられた時に
    1. ワンホット 整数表現に符号化
    2. ワンホット 整数表現をその文字出力に復号化
    3. 出力として得られた確率ベクトルを，対応する文字に復号化
    """
    def __init__(self, chars):
        """文字表の初期化

        引数
            chars: 入力信号に現れる全文字集合
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """文字列 C のワンホット表現

        引数
            num_rows: ワンホット表現ベクトルを返す行数
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [4]:
class colors:
    ok = '\033[92m'    # 正解表示用の色を指定
    fail = '\033[91m'  # 失敗表示用の色を指定
    close = '\033[0m'  # 表示色をデフォルトに戻す

# 訓練に用いるパラメータ
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

# 入力の最大長は `数字+数字` で表す
MAXLEN = DIGITS + 1 + DIGITS

# 入力に用いる文字と埋め草文字(空白)を指定
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [5]:
questions = []
expected = []
seen = set()
print('データの生成...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()

    # 既知の足し算問題をスキップ
    # 同様に x+Y == Y+x のような問題もスキップ。ソートもする。
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)

    # 常にMAXLENになるようにデータを空白で埋める
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # 回答 `ans` は最大で DIGITS + 1 のサイズ
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # 例えば '12+345 ' は ' 543+21' となる。
        # 埋め草文字の空白注意
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print(f'総問題数: {len(questions)}')

データの生成...
総問題数: 50000


In [7]:
print('ベクトル化...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool_)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool_)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

ベクトル化...


In [8]:
# x の後半部分はほとんど大きな数字になるためデータ (x, y) をシャッフル
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [9]:
# データセットの 10% を検証データセットとして設定。検証データセットでは学習を行わない
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('訓練データセット数:')
print(f'入力データサイズ: {x_train.shape}')
print(f'教師データサイズ:{y_train.shape}')

print('検証データセット数:')
print(f'入力データサイズ: {x_val.shape}')
print(f'教師データサイズ: {y_val.shape}')

訓練データセット数:
入力データサイズ: (45000, 7, 12)
教師データサイズ:(45000, 4, 12)
検証データセット数:
入力データサイズ: (5000, 7, 12)
教師データサイズ: (5000, 4, 12)


In [10]:
# GRU や SimpleRNN を置き換えて実行してみること
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [11]:
print('モデル作成...')
model = Sequential()

# RNN を使って入力配列を符号化し，HIDDEN_SIZE の出力を生成
# 注：入力系列が可変長の場合には `input_shape=(None, num_feature)` を使用
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# 復号化器 RNN の入力として，最終時刻の隠れ層の状態を繰り返し与える。
# RNN の最終時刻の隠れ層の状態を，各時間ステップで繰り返し提供。
# すなわち，DIGITS + 1 回繰り返す。
# 例えば DIGITS=3 の場合，最大出力は999+999=1998 となる。
model.add(layers.RepeatVector(DIGITS + 1))

# 復号化器 RNN は，複数層を重ねることも，単層にすることも可能
for _ in range(LAYERS):
    # `return_sequences=True` にすると，最後の出力だけでなく，これまでの全出力を (num_samples, timesteps, output_dim) という形で返す。
    # これは，下記  `TimeDistributed` が最初の次元を `timesteps` と想定しているため，必要となる。
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# 入力の各時間スライスに全結合層を適用する。
# 出力系列の各時刻に対して，どの文字を選択するかを決定するため
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

モデル作成...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        72,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 4, 128)         │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 4, 12)          │         1,548 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 4, 12)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,324 (802.05 KB)

 Trainable params: 205,324 (802.05 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# モデルをエポックごとに学習させ，検証用データセットに対する予測値を表示
epochs = 30
n_sample = 10
for epoch in range(epochs):
    print()
    print('-' * 78)
    print(f'エポック数: {epoch+1}')
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1, validation_data=(x_val, y_val))

    # 検証データセットから n_sample 個の事例をランダムサンプリングして結果を表示
    for i in range(n_sample):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]

        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('問:', q[::-1] if INVERT else q)
        print('答:', correct)
        if correct == guess:
            print(colors.ok + '正' + colors.close, end=" ")
        else:
            print(colors.fail + '誤' + colors.close, end=" ")
        print(guess)
        print('---')


------------------------------------------------------------------------------
エポック数: 1
352/352 ━━━━━━━━━━━━━━━━━━━━ 16s 35ms/step - accuracy: 0.2891 - loss: 2.0188 - val_accuracy: 0.3428 - val_loss: 1.7795
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
問: 3+722  
答: 725 
誤 33  
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
問: 4+807  
答: 811 
誤 121 
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
問: 117+620
答: 737 
誤 122 
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
問: 834+905
答: 1739
誤 1123
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
問: 419+706
答: 1125
誤 102 
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
問: 334+68 
答: 402 
誤 123 
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
問: 305+4  
答: 309 
誤 42  
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
問: 47+941 
答: 988 
誤 121 
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
問: 70+44  
答: 114 
誤 121 
---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
問: 161+71 
答: 232 
誤 121 
---

------------------------------------------------------------------------------
エポック数: 2
352/352 ━━━━━━━━━━━━━━